In [1]:
import pandas as pd

In [2]:
df_eacl_mal_train = pd.read_csv("../data/mal data/mal_full_offensive_train.csv",encoding="utf-8",sep="\t",usecols=[0,1])
df_eacl_mal_dev = pd.read_csv("../data/mal data/mal_full_offensive_dev.csv",encoding="utf-8",sep="\t",usecols=[0,1])
df_eacl_mal_test = pd.read_csv("../data/mal data/mal_full_offensive_test_with_labels.csv",encoding="utf-8",sep="\t",usecols=[0,1])

df_eacl_mal_train = df_eacl_mal_train.dropna()
df_eacl_mal_dev = df_eacl_mal_dev.dropna()
df_eacl_mal_test = df_eacl_mal_test.dropna()

print(df_eacl_mal_test.info())

eacl_mal_train_sentences = list(df_eacl_mal_train['text'].values)
eacl_mal_train_labels = list(df_eacl_mal_train['label'].values)

eacl_mal_dev_sentences = list(df_eacl_mal_dev['text'].values)
eacl_mal_dev_labels = list(df_eacl_mal_dev['label'].values)

eacl_mal_test_sentences = list(df_eacl_mal_test.values[:,0])
eacl_mal_test_labels = list(df_eacl_mal_test.values[:,1])

x_train = eacl_mal_train_sentences
y_train = eacl_mal_train_labels

x_dev = eacl_mal_dev_sentences
y_dev = eacl_mal_dev_labels

x_test = eacl_mal_test_sentences
y_test = eacl_mal_test_labels

print(len(x_train),len(x_dev),len(x_test)," total = ",len(x_train)+len(x_dev)+len(x_test))

print(set(eacl_mal_train_labels))
print(set(eacl_mal_dev_labels))
print(set(eacl_mal_test_labels))

total_sentences = []
total_sentences.extend(eacl_mal_train_sentences)
total_sentences.extend(eacl_mal_dev_sentences)
total_sentences.extend(eacl_mal_test_sentences)

total_labels = []
total_labels.extend(eacl_mal_train_labels)
total_labels.extend(eacl_mal_dev_labels)
total_labels.extend(eacl_mal_test_labels)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2000 entries, 0 to 1999
Data columns (total 2 columns):
 #   Column                                                       Non-Null Count  Dtype 
---  ------                                                       --------------  ----- 
 0   അപ്പൊ ഇതൊരൊന്നൊരാ മൊതലാണല്ലേ  Suraj ആണ് നടൻ ന്ന് തോന്നുന്നു  2000 non-null   object
 1   Not_offensive                                                2000 non-null   object
dtypes: object(2)
memory usage: 46.9+ KB
None
16010 1999 2000  total =  20009
{'Not_offensive', 'not-malayalam', 'Offensive_Targeted_Insult_Group', 'Offensive_Untargetede', 'Offensive_Targeted_Insult_Individual'}
{'Not_offensive', 'not-malayalam', 'Offensive_Untargetede', 'Offensive_Targeted_Insult_Individual', 'Offensive_Targeted_Insult_Group'}
{'Not_offensive', 'not-malayalam', 'Offensive_Untargetede', 'Offensive_Targeted_Insult_Individual', 'Offensive_Targeted_Insult_Group'}


In [3]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(total_sentences)
print(X.shape)

(20009, 23826)


In [4]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
cosine_sim = cosine_similarity(X,X)

In [5]:
sum_rows_cosine_sim = np.sum(cosine_sim,axis=1)

In [6]:
dict_cosine_sim = {}
for i in range(sum_rows_cosine_sim.shape[0]):
    dict_cosine_sim[i] = sum_rows_cosine_sim[i]


In [7]:
sorted_dict = dict(sorted(dict_cosine_sim.items(), key=lambda item: item[1],reverse=True))

In [8]:
dict_cosine_sim = {}
for i in range(len(y_train)+len(y_dev),sum_rows_cosine_sim.shape[0]):
    dict_cosine_sim[i] = sum_rows_cosine_sim[i]

In [9]:
sorted_dict = dict(sorted(dict_cosine_sim.items(), key=lambda item: item[1],reverse=True))

In [10]:
from itertools import islice

def take(n, iterable):
    "Return first n items of the iterable as a list"
    return list(islice(iterable, n))

n_items = take(200, sorted_dict.items())

In [11]:
print(n_items)

[(19305, 515.699381754824), (19457, 515.699381754824), (19820, 515.699381754824), (18291, 461.41674303750847), (18385, 457.74798046294075), (18981, 456.54222355278694), (18626, 443.6476614538725), (18614, 442.1282699389809), (18886, 441.8608997591772), (19486, 438.40739553512566), (18480, 436.4534270056293), (19525, 434.2763797907662), (18525, 432.42538480989776), (19227, 426.34841896660305), (18294, 423.43437328341173), (19218, 419.4509302164013), (18422, 406.24212923471066), (18124, 405.2668270501923), (18236, 405.2668270501923), (18920, 405.2668270501923), (19152, 397.48110660530165), (19335, 396.7054869603914), (18101, 396.00357511612935), (19367, 396.00357511612935), (18826, 395.38359201928165), (19720, 393.15309401165223), (18552, 390.0846980411259), (19671, 389.0486326696663), (19761, 388.6577041715983), (18214, 387.25437955013217), (18239, 384.30125409656506), (18798, 382.67380808914015), (18446, 382.3174327329308), (18321, 380.61607813663454), (19613, 379.2595968146383), (1904

In [12]:
dict_key_dissmilar={}

for key,value in n_items:
    similar_values = cosine_sim[key]
    count_dissimilar_labels = 0
    count_similar_labels=0
    key_label = total_labels[key]
    for i in range(len(similar_values)):
        if cosine_sim[key][i]>0:
            if key_label == total_labels[i]:
                count_similar_labels+=1
            else:
                count_dissimilar_labels+=1
        diff = count_dissimilar_labels-count_similar_labels
        dict_key_dissmilar[key] = diff

In [13]:
sorted_key_dict = dict(sorted(dict_key_dissmilar.items(), key=lambda item: item[1],reverse=True))
print(sorted_key_dict)

{18321: 3671, 19798: 3279, 19460: 2234, 18594: 2002, 18839: 1973, 19527: 1885, 19280: 1549, 19255: 1453, 19677: -1250, 19063: -1411, 19782: -1411, 19542: -1473, 19232: -1693, 18944: -1812, 19511: -1820, 19448: -1827, 19656: -1834, 19908: -1850, 19104: -1860, 18276: -1866, 18284: -1953, 18759: -1970, 19598: -1980, 19356: -2028, 19026: -2035, 18494: -2057, 19267: -2076, 18268: -2136, 19568: -2138, 18935: -2139, 19360: -2177, 18235: -2192, 18107: -2215, 18239: -2223, 19814: -2231, 18129: -2267, 19882: -2268, 19042: -2270, 18515: -2276, 19627: -2285, 18255: -2286, 19442: -2301, 19920: -2347, 19083: -2364, 19179: -2376, 18487: -2394, 19158: -2396, 18102: -2412, 19191: -2414, 19305: -2428, 19457: -2428, 19820: -2428, 18886: -2446, 19751: -2469, 19287: -2470, 18974: -2475, 19873: -2483, 18602: -2487, 18606: -2499, 18422: -2531, 19670: -2541, 19290: -2553, 18496: -2559, 18726: -2602, 19227: -2622, 18728: -2642, 19899: -2642, 18474: -2658, 19957: -2673, 19588: -2674, 19522: -2677, 19746: -2683,

In [14]:
n_key_items = take(100, sorted_key_dict.items())

In [15]:
from collections import Counter
dict_key_dissmilar_label_count={}
for key,value in n_key_items:
    dissimilar_labels=[]
    similar_values = cosine_sim[key]
    key_label = total_labels[key]
    if value>0:
        for i in range(len(similar_values)):
            if cosine_sim[key][i]>0:
                if key_label != total_labels[i]:
                    dissimilar_labels.append(total_labels[i])
        counted = Counter(dissimilar_labels)
        most_common_counts = counted.most_common()
        dict_key_dissmilar_label_count[key] = most_common_counts
    
for key,value in dict_key_dissmilar_label_count.items():
    print(key,value)

18321 [('Not_offensive', 3665), ('Offensive_Untargetede', 33), ('Offensive_Targeted_Insult_Group', 32)]
19798 [('Not_offensive', 3278), ('Offensive_Untargetede', 38), ('Offensive_Targeted_Insult_Group', 31)]
19460 [('Not_offensive', 2607), ('Offensive_Targeted_Insult_Group', 45), ('Offensive_Untargetede', 23), ('Offensive_Targeted_Insult_Individual', 9)]
18594 [('Not_offensive', 2349), ('Offensive_Targeted_Insult_Group', 39), ('Offensive_Untargetede', 22), ('Offensive_Targeted_Insult_Individual', 10)]
18839 [('Not_offensive', 2332), ('Offensive_Targeted_Insult_Group', 39), ('Offensive_Untargetede', 25), ('Offensive_Targeted_Insult_Individual', 19)]
19527 [('Not_offensive', 2337), ('Offensive_Targeted_Insult_Group', 41), ('Offensive_Untargetede', 28), ('Offensive_Targeted_Insult_Individual', 21)]
19280 [('Not_offensive', 2012), ('Offensive_Untargetede', 7), ('Offensive_Targeted_Insult_Group', 6), ('Offensive_Targeted_Insult_Individual', 6)]
19255 [('Not_offensive', 1951), ('Offensive_Ta

In [18]:
qa_data = []
qa_labels=[]
for key,value in n_key_items:
    if value >0:
        qa_data.append(total_sentences[key])
        qa_labels.append(total_labels[key])
    
qa_df = pd.DataFrame(zip(qa_data,qa_labels),columns=['qa_data','qa_labels'])
qa_df.to_csv('malayalam_eacl_qa.tsv',sep='\t',index=None)